# Imports

In [0]:
import pandas as pd 
import requests
import io

# Load Data

In [14]:
BASE_URL = "https://docs.google.com/spreadsheets/d/1VpLXByvbTfbgupDNWcUaBxdk72cW9FRExs0AgRVaF1g/export?format=csv&"
URL_DEMOGRAPHICS = BASE_URL+"gid=611260429"
URL_PREFERENCES = BASE_URL+"gid=961690077"
content_demographics = requests.get(URL_DEMOGRAPHICS, verify=False).content
content_preference = requests.get(URL_PREFERENCES, verify=False).content
df_demographics = pd.read_csv(io.StringIO(content_demographics.decode('utf-8'))) 
df_preference = pd.read_csv(io.StringIO(content_preference.decode('utf-8'))) 

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [15]:
df_demographics.head(1)

,ParticipantID,Age,Gender,DominantHand,AvgTimeSpent,WordsPerMinute,WordCount,Errors,TimeTakenS,TimeTakenM,MeanError,totalwords,time,sec
0,participant_1,27.0,Male,Right,10.0,57.0,89.0,15.0,94.0,1.561404,9.606742,104.0,1.561404,93.684211


In [13]:
df_preference.head(1)

,Participant Id,Comfort_Below_Left_Fist_Down,Preference_Below_Left_Fist_Down,Comfort_Below_Right_Fist_Down,Preference_Below_Right_Fist_Down,Comfort_Below_Left_Fist_Side,Preference_Below_Left_Fist_Side,Comfort_Below_Right_Fist_Side,Preference_Below_Right_Fist_Side,Comfort_Below_Left_Fist_Up,Preference_Below_Left_Fist_Up,Comfort_Below_Right_Fist_Up,Preference_Below_Right_Fist_Up,Comfort_Below_Left_Palm_Down,Preference_Below_Left_Palm_Down,Comfort_Below_Right_Palm_Down,Preference_Below_Right_Palm_Down,Comfort_Below_Left_Palm_Side,Preference_Below_Left_Palm_Side,Comfort_Below_Right_Palm_Side,Preference_Below_Right_Palm_Side,Comfort_Below_Left_Palm_Up,Preference_Below_Left_Palm_Up,Comfort_Below_Right_Palm_Up,Preference_Below_Right_Palm_Up,Comfort_On_Left_Fist_Down,Preference_On_Left_Fist_Down,Comfort_On_Right_Fist_Down,Preference_On_Right_Fist_Down,Comfort_On_Left_Fist_Side,Preference_On_Left_Fist_Side,Comfort_On_Right_Fist_Side,Preference_On_Right_Fist_Side,Comfort_On_Left_Fist_Up,Preference_On_Left_Fist_Up,Comfort_On_Right_Fist_Up,Preference_On_Right_Fist_Up,Comfort_On_Left_Palm_Down,Preference_On_Left_Palm_Down,Comfort_On_Right_Palm_Down,Preference_On_Right_Palm_Down,Comfort_On_Left_Palm_Side,Preference_On_Left_Palm_Side,Comfort_On_Right_Palm_Side,Preference_On_Right_Palm_Side,Comfort_On_Left_Palm_Up,Preference_On_Left_Palm_Up,Comfort_On_Right_Palm_Up,Preference_On_Right_Palm_Up,Comfort_Beside_Left_Fist_Down,Preference_Beside_Left_Fist_Down,Comfort_Beside_Right_Fist_Down,Preference_Beside_Right_Fist_Down,Comfort_Beside_Left_Fist_Side,Preference_Beside_Left_Fist_Side,Comfort_Beside_Right_Fist_Side,Preference_On_Right_Fist_Side.1,Comfort_Beside_Left_Fist_Up,Preference_Beside_Left_Fist_Up,Comfort_Beside_Right_Fist_Up,Preference_Beside_Right_Fist_Up,Comfort_Beside_Left_Palm_Down,Preference_Beside_Left_Palm_Down,Comfort_Beside_Right_Palm_Down,Preference_Beside_Right_Palm_Down,Comfort_Beside_Left_Palm_Side,Preference_Beside_Left_Palm_Side,Comfort_Beside_Right_Palm_Side,Preference_Beside_Right_Palm_Side,Comfort_Beside_Left_Palm_Up,Preference_Beside_Left_Palm_Up,Comfort_Beside_Right_Palm_Up,Preference_Beside_Right_Palm_Up
0,participant_1,3,2,4,4,2,2,4,4,1,1,2,2,3,3,5,5,3,3,4,4,1,1,2,2,2,2,4,4,1,1,4,4,1,1,2,2,3,3,4,4,2,2,4,4,1,1,1,1,2,2,3,3,2,2,4,4.0,1,1,1,1,3,3,4,4,3,3,4,4,1,1,1,1


# Analysis

## Demographics

In [3]:
# GENDER DISTRIBUTION
data.Gender.value_counts()

Male      20
Female    10
Name: Gender, dtype: int64

In [4]:
# AVERAGE AGE
print("MEAN: ",data["Age"].mean())
print("STD: ",data["Age"].std())

MEAN:  24.566666666666666
STD:  2.4023455970813385


In [6]:
# AVERAGE USAGE
print("MEAN: ",data["AvgTimeSpent"].mean())
print("STD: ",data["AvgTimeSpent"].std())

MEAN:  7.0
STD:  3.723179953415795


In [5]:
# AVERAGE TYPING SPEED
print("MEAN: ",data["WordsPerMinute"].mean())
print("STD: ",data["WordsPerMinute"].std())

MEAN:  34.9
STD:  9.531617078855561


In [7]:
# AVERAGE ERROR RATE
print("MEAN: ",data["MeanError"].mean())
print("STD: ",data["MeanError"].std())

MEAN:  5.735205992738709
STD:  2.6533760931418673


## Preference

In [0]:
df_preference_mean = df_preference.mean()

In [43]:
df_preference_mean.to_frame().transpose()

,Comfort_Below_Left_Fist_Down,Preference_Below_Left_Fist_Down,Comfort_Below_Right_Fist_Down,Preference_Below_Right_Fist_Down,Comfort_Below_Left_Fist_Side,Preference_Below_Left_Fist_Side,Comfort_Below_Right_Fist_Side,Preference_Below_Right_Fist_Side,Comfort_Below_Left_Fist_Up,Preference_Below_Left_Fist_Up,Comfort_Below_Right_Fist_Up,Preference_Below_Right_Fist_Up,Comfort_Below_Left_Palm_Down,Preference_Below_Left_Palm_Down,Comfort_Below_Right_Palm_Down,Preference_Below_Right_Palm_Down,Comfort_Below_Left_Palm_Side,Preference_Below_Left_Palm_Side,Comfort_Below_Right_Palm_Side,Preference_Below_Right_Palm_Side,Comfort_Below_Left_Palm_Up,Preference_Below_Left_Palm_Up,Comfort_Below_Right_Palm_Up,Preference_Below_Right_Palm_Up,Comfort_On_Left_Fist_Down,Preference_On_Left_Fist_Down,Comfort_On_Right_Fist_Down,Preference_On_Right_Fist_Down,Comfort_On_Left_Fist_Side,Preference_On_Left_Fist_Side,Comfort_On_Right_Fist_Side,Preference_On_Right_Fist_Side,Comfort_On_Left_Fist_Up,Preference_On_Left_Fist_Up,Comfort_On_Right_Fist_Up,Preference_On_Right_Fist_Up,Comfort_On_Left_Palm_Down,Preference_On_Left_Palm_Down,Comfort_On_Right_Palm_Down,Preference_On_Right_Palm_Down,Comfort_On_Left_Palm_Side,Preference_On_Left_Palm_Side,Comfort_On_Right_Palm_Side,Preference_On_Right_Palm_Side,Comfort_On_Left_Palm_Up,Preference_On_Left_Palm_Up,Comfort_On_Right_Palm_Up,Preference_On_Right_Palm_Up,Comfort_Beside_Left_Fist_Down,Preference_Beside_Left_Fist_Down,Comfort_Beside_Right_Fist_Down,Preference_Beside_Right_Fist_Down,Comfort_Beside_Left_Fist_Side,Preference_Beside_Left_Fist_Side,Comfort_Beside_Right_Fist_Side,Preference_On_Right_Fist_Side.1,Comfort_Beside_Left_Fist_Up,Preference_Beside_Left_Fist_Up,Comfort_Beside_Right_Fist_Up,Preference_Beside_Right_Fist_Up,Comfort_Beside_Left_Palm_Down,Preference_Beside_Left_Palm_Down,Comfort_Beside_Right_Palm_Down,Preference_Beside_Right_Palm_Down,Comfort_Beside_Left_Palm_Side,Preference_Beside_Left_Palm_Side,Comfort_Beside_Right_Palm_Side,Preference_Beside_Right_Palm_Side,Comfort_Beside_Left_Palm_Up,Preference_Beside_Left_Palm_Up,Comfort_Beside_Right_Palm_Up,Preference_Beside_Right_Palm_Up
0,3.7,3.466667,3.933333,3.733333,3.866667,3.666667,4.3,4.2,2.133333,2.3,2.5,2.5,4.133333,3.933333,4.333333,4.3,3.666667,3.533333,3.966667,3.7,2.366667,2.466667,2.3,2.433333,3.433333,3.366667,3.666667,3.533333,3.366667,3.333333,3.766667,3.6,1.9,2.1,2.166667,2.366667,4.066667,3.933333,4.366667,4.366667,3.5,3.366667,3.766667,3.8,2.2,2.3,2.333333,2.366667,3.233333,2.933333,3.4,3.2,3.5,3.033333,3.733333,3.241379,1.8,2.033333,2.066667,2.033333,3.866667,3.4,4.033333,3.566667,3.566667,3.333333,3.866667,3.533333,2.066667,2.066667,2.166667,2.233333


In [53]:
df_preference_mean.sort_values().head(36)

Comfort_Beside_Left_Fist_Up          1.800000
Comfort_On_Left_Fist_Up              1.900000
Preference_Beside_Right_Fist_Up      2.033333
Preference_Beside_Left_Fist_Up       2.033333
Preference_Beside_Left_Palm_Up       2.066667
Comfort_Beside_Left_Palm_Up          2.066667
Comfort_Beside_Right_Fist_Up         2.066667
Preference_On_Left_Fist_Up           2.100000
Comfort_Below_Left_Fist_Up           2.133333
Comfort_On_Right_Fist_Up             2.166667
Comfort_Beside_Right_Palm_Up         2.166667
Comfort_On_Left_Palm_Up              2.200000
Preference_Beside_Right_Palm_Up      2.233333
Preference_Below_Left_Fist_Up        2.300000
Comfort_Below_Right_Palm_Up          2.300000
Preference_On_Left_Palm_Up           2.300000
Comfort_On_Right_Palm_Up             2.333333
Preference_On_Right_Palm_Up          2.366667
Comfort_Below_Left_Palm_Up           2.366667
Preference_On_Right_Fist_Up          2.366667
Preference_Below_Right_Palm_Up       2.433333
Preference_Below_Left_Palm_Up     

In [54]:
df_preference_mean.sort_values().tail(36)

Comfort_On_Left_Fist_Down            3.433333
Preference_Below_Left_Fist_Down      3.466667
Comfort_On_Left_Palm_Side            3.500000
Comfort_Beside_Left_Fist_Side        3.500000
Preference_On_Right_Fist_Down        3.533333
Preference_Below_Left_Palm_Side      3.533333
Preference_Beside_Right_Palm_Side    3.533333
Comfort_Beside_Left_Palm_Side        3.566667
Preference_Beside_Right_Palm_Down    3.566667
Preference_On_Right_Fist_Side        3.600000
Comfort_On_Right_Fist_Down           3.666667
Comfort_Below_Left_Palm_Side         3.666667
Preference_Below_Left_Fist_Side      3.666667
Comfort_Below_Left_Fist_Down         3.700000
Preference_Below_Right_Palm_Side     3.700000
Comfort_Beside_Right_Fist_Side       3.733333
Preference_Below_Right_Fist_Down     3.733333
Comfort_On_Right_Fist_Side           3.766667
Comfort_On_Right_Palm_Side           3.766667
Preference_On_Right_Palm_Side        3.800000
Comfort_Beside_Right_Palm_Side       3.866667
Comfort_Below_Left_Fist_Side      